In [2]:
from mbrlax.models import SVGP, GPModelSpec, initialize_gp_model
from gpjax.likelihoods import Gaussian, Likelihood
from evosax import Strategy, CMA_ES
import jax.numpy as jnp
import tensorflow as tf
from tensorflow_probability.python import bijectors
from gpflow.config import default_float
from mbrlax.utils import Driver
from bsuite.environments import cartpole
import jax

In [3]:
def random_policy(key):
    return jax.random.choice(key=key, a=action_space)

def generate_experience(env, key, num_steps=30):
    time_step = env.reset()
    states, actions = [], []
    states.append(time_step.observation[0])
    for step in range(num_steps):
        action = random_policy(key)
        actions.append([action])
        time_step = env.step(action)
        if time_step.last(): time_step = env.reset()
        states.append(time_step.observation[0])

    return jnp.asarray(states), jnp.asarray(actions)

def get_gp_data(states, actions):
    return states, actions

In [4]:
def populate_replay_buffer(env, key, num_steps=1000):
    time_step = env.reset()
    print(type(time_step.observation))
    replay_buffer.push(time_step, None)
    for step in range(num_steps):
        action = random_policy(key)
        time_step = env.step(action)
        replay_buffer.push(env_output=time_step, action=action)
        if time_step.last(): time_step = env.reset()

In [5]:
seed = 42
replay_buffer.reset()
key = jax.random.PRNGKey(seed)
env = cartpole.Cartpole(seed=seed)
populate_replay_buffer(env=env, key=key)
obs_tm1, a_tm1, r_t, discount_t, obs_t = replay_buffer.sample(100)

assert(obs_tm1.shape == (100, 6))
assert(a_tm1.shape == (100,))
assert(discount_t.shape == (100,))
assert(r_t.shape == (100,))
assert(obs_t.shape == (100,6))

obs_tm1, a_tm1, r_t, discount_t, obs_t = replay_buffer.gather_all()
assert(obs_tm1.shape == (1000, 6))
assert(a_tm1.shape == (1000,))
assert(discount_t.shape == (1000,))
assert(r_t.shape == (1000,))
assert(obs_t.shape == (1000,6))s

SyntaxError: invalid syntax (2583288525.py, line 19)

In [ ]:
seed = 42
action_space = jnp.array([0,1,2])
key = jax.random.PRNGKey(seed)
env = cartpole.Cartpole(seed=seed)
inputs, targets = generate_experience(env=env, key=key)
inputs.shape, targets.shape

((31, 6), (30, 1))

In [ ]:
class RandomPolicy:
    def step(self, time_step):
        return 

In [ ]:
# params = transition_model.model.get_params()
# print(len(params["kernel"]))
# print(params["kernel"][1]["lengthscales"].shape)
# print(params["kernel"][1]["variance"])
# print(params["likelihood"]["variance"].shape)
# print(params["inducing_variable"].shape)
# print(params["q_mu"].shape)
# print(params["q_sqrt"].shape)

In [ ]:
real_collect_driver = Driver(
    mode="random",
    env=self.env,
    policy=policy,
    transition_observers=[self.agent.real_replay_buffer.push],
    observers=[episode_metrics],
    max_steps=31
)

NameError: name 'self' is not defined

In [ ]:
# inverse link function
invlink = bijectors.Chain(bijectors=[
    bijectors.Scale(scale=tf.cast(x=20 - 1e-5, dtype=default_float())),
    bijectors.Shift(shift=tf.cast(x=-0.5, dtype=default_float())),
    bijectors.NormalCDF()]
)

model_spec = GPModelSpec(
    type=SVGP,
    num_inducing=32,
    likelihood=Gaussian(),
    prior=None,
    mean_function="default",
    model_uncertainty=False,
    invlink = invlink
)
policy = initialize_gp_model(data=(inputs, targets), model_spec=model_spec)

dict_keys(['kernel', 'likelihood', 'mean_function', 'inducing_variable', 'q_mu', 'q_sqrt'])


In [ ]:
print(model_parmas["kernel"][0]["lengthscales"].shape)
print(model_parmas["inducing_variable"].shape)
print(model_parmas["q_mu"].shape)
print(model_parmas["likelihood"]["variance"].shape)

(6,)
(31, 6)
(31, 1)
(1,)
